# Project 3

In this project, I will use the data from University of Dayton on daily temperature level of major cities. The aim is to analysis a temporal change of average temperatures around the world. The key hypothesis is that there is a significant trend of higher average temperature, at the same time of year, that can be seen in countries around the world.

- **Dataset(s) to be used:** [[(https://www.kaggle.com/datasets/sudalairajkumar/daily-temperature-of-major-cities)]
- **Analysis question:** ["Is there a significant trend of higher average temperature, at the same time of year, that can be seen in countries around the world?"]
- **Columns that will (likely) be used:**
  - [city_temperature][Country]
  - [city_temperature][Year] 
  - [city_temperature][Month]
  - [city_temperature][AvgTemperature] 
- (If you're using multiple datasets) **Columns to be used to merge/join them:**
  - [uncode] [name]
  - [uncode] [alpha-3]
- **Hypothesis**: [There is a significant trend of higher average temperature, at the same time of year, that can be seen in countries around the world]
- **Site URL:** [the `https://computingincontext.readthedocs.io/en/latest/index.html` URL of your live site, from the Publish section]

In [82]:
import pandas as pd
import plotly.express as px
import numpy as np

In [83]:
import plotly.io as pio
pio.renderers.default = "vscode+jupyterlab+notebook_connected"

Read in data, including the main file and the UN country code file, of which alpha-3 code will be used for plotly's chrolopeth map

In [84]:
all_temp = pd.read_csv("city_temperature.csv")
all_temp.head()

C:\Users\patip\AppData\Local\Temp\ipykernel_17372\1442917994.py:1: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [85]:
uncode = pd.read_csv("uncode.csv")
uncode.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [86]:
uncode = uncode[['name', 'alpha-3']]
uncode.head()

,name,alpha-3
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [87]:
all_temp = pd.merge(all_temp, uncode[['name', 'alpha-3']], left_on='Country', right_on='name', how='left')

all_temp = all_temp.drop(columns=['name'])

cols = ['Region', 'alpha-3', 'Country', 'State', 'City', 'Month', 'Day', 'Year', 'AvgTemperature']
all_temp = df_merged[cols]

print(all_temp)

                Region alpha-3  Country                   State  \
0               Africa     DZA  Algeria                     NaN   
1               Africa     DZA  Algeria                     NaN   
2               Africa     DZA  Algeria                     NaN   
3               Africa     DZA  Algeria                     NaN   
4               Africa     DZA  Algeria                     NaN   
...                ...     ...      ...                     ...   
2906322  North America     NaN       US  Additional Territories   
2906323  North America     NaN       US  Additional Territories   
2906324  North America     NaN       US  Additional Territories   
2906325  North America     NaN       US  Additional Territories   
2906326  North America     NaN       US  Additional Territories   

                         City  Month  Day  Year  AvgTemperature  
0                     Algiers      1    1  1995            64.2  
1                     Algiers      1    2  1995            49.4

Checking data type of each column

In [88]:
print(all_temp.dtypes)

Region             object
alpha-3            object
Country            object
State              object
City               object
Month               int64
Day                 int64
Year                int64
AvgTemperature    float64
dtype: object


Because the analysis will focus on country level, as well as monthly data to adjust for seasonality, we dropped other columns.

In [89]:

df_grouped = all_temp.groupby(['Country', 'Month', 'Year'])['AvgTemperature'].mean().reset_index()

df_grouped

,Country,Month,Year,AvgTemperature
0,Albania,1,1995,-99.000000
1,Albania,1,1996,-99.000000
2,Albania,1,1997,-99.000000
3,Albania,1,1998,-99.000000
4,Albania,1,1999,-5.219355
...,...,...,...,...
36280,Zambia,12,2009,67.270968
36281,Zambia,12,2010,64.267742
36282,Zambia,12,2011,74.654839
36283,Zambia,12,2012,73.319355


From the table, we can see the incomplete time series information, i.e. -99 value for AvgTemperature, which should be dropped before analysis

In [90]:
df_grouped_sorted = df_grouped.sort_values(by=['Year', 'Month'], ascending=[True, True]).reset_index(drop=True)
df_grouped_sorted = df_grouped_sorted[df_grouped_sorted['AvgTemperature'] != -99]
df_grouped_sorted

,Country,Month,Year,AvgTemperature
10,Algeria,1,1995,51.419355
11,Argentina,1,1995,73.351613
12,Australia,1,1995,71.871613
13,Austria,1,1995,31.867742
14,Bahamas,1,1995,71.509677
...,...,...,...,...
36280,Uruguay,5,2020,57.246154
36281,Uzbekistan,5,2020,70.307692
36282,Venezuela,5,2020,80.225000
36283,Vietnam,5,2020,82.815385


Also select only data from years of interest (2000 - 2011)

In [91]:

df_filtered = df_grouped_sorted[df_grouped_sorted['Year'].between(2000, 2011)]


print(df_filtered)

          Country  Month  Year  AvgTemperature
7474      Albania      1  2000       30.861290
7475      Algeria      1  2000       47.235484
7476    Argentina      1  2000       77.641935
7477    Australia      1  2000       69.763226
7478      Austria      1  2000       29.687097
...           ...    ...   ...             ...
25031  Uzbekistan     12  2011       30.993548
25032   Venezuela     12  2011       78.129032
25033     Vietnam     12  2011       62.916129
25034  Yugoslavia     12  2011       40.303226
25035      Zambia     12  2011       74.654839

[17183 rows x 4 columns]


Turning the Year and Month column into datetime for time series analysis

In [92]:


df_filtered['Date'] = pd.to_datetime(df_filtered[['Year', 'Month']].assign(DAY=1))

df_filtered = df_filtered.drop(columns=['Year', 'Month'])

print(df_filtered)


          Country  AvgTemperature       Date
7474      Albania       30.861290 2000-01-01
7475      Algeria       47.235484 2000-01-01
7476    Argentina       77.641935 2000-01-01
7477    Australia       69.763226 2000-01-01
7478      Austria       29.687097 2000-01-01
...           ...             ...        ...
25031  Uzbekistan       30.993548 2011-12-01
25032   Venezuela       78.129032 2011-12-01
25033     Vietnam       62.916129 2011-12-01
25034  Yugoslavia       40.303226 2011-12-01
25035      Zambia       74.654839 2011-12-01

[17183 rows x 3 columns]


C:\Users\patip\AppData\Local\Temp\ipykernel_17372\2244757008.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Convert Fahrenheit to Celsius

In [101]:
# Convert AvgTemperature from Fahrenheit to Celsius
df_filtered['AvgTemperature'] = (df_filtered['AvgTemperature'] - 32) * 5 / 9

# Display the DataFrame with the converted temperature
print(df_filtered)


          Country  AvgTemperature       Date
7475      Algeria      -13.075468 2000-01-01
7476    Argentina       -3.690761 2000-01-01
7477    Australia       -6.122461 2000-01-01
7478      Austria      -18.491637 2000-01-01
7479      Bahamas       -5.976703 2000-01-01
...           ...             ...        ...
25030     Uruguay       -6.633811 2011-12-01
25031  Uzbekistan      -18.088411 2011-12-01
25032   Venezuela       -3.540422 2011-12-01
25033     Vietnam       -8.235763 2011-12-01
25034  Yugoslavia      -15.215054 2011-12-01

[12224 rows x 3 columns]


Further list of countries with incomplete information, created using the Excel spreadsheet 

In [ ]:

countries_to_remove = [
    'Burundi', 'Georgia', 'Cyprus', 'Kenya', 'Malawi', 'Ethiopia', 'Gambia', 
    'Nepal', 'Suriname', 'Guyana', 'Indonesia', 'Guinea-Bissau', 'Haiti', 
    'Guinea', 'Ivory Coast', 'Sri Lanka', 'Gabon', 'Mozambique', 'Oman', 
    'Equador', 'Bangladesh', 'Germany', 'Tajikistan', 'North Korea', 
    'Tanzania', 'Uganda', 'Sierra Leone', 'Taiwan', 'Namibia', 'Togo', 
    'Benin', 'Albania', 'Pakistan', 'Barbados', 'Mongolia', 'Nigeria', 
    'Zambia', 'Central African Republic', 'Madagascar'
]

df_filtered = df_filtered[~df_filtered['Country'].isin(countries_to_remove)]


print(df_filtered)


          Country  AvgTemperature       Date
7475      Algeria        8.464158 2000-01-01
7476    Argentina       25.356631 2000-01-01
7477    Australia       20.979570 2000-01-01
7478      Austria       -1.284946 2000-01-01
7479      Bahamas       21.241935 2000-01-01
...           ...             ...        ...
25030     Uruguay       20.059140 2011-12-01
25031  Uzbekistan       -0.559140 2011-12-01
25032   Venezuela       25.627240 2011-12-01
25033     Vietnam       17.175627 2011-12-01
25034  Yugoslavia        4.612903 2011-12-01

[12224 rows x 3 columns]


Re-merge the alpha-3 code

In [ ]:
all_temp_filtered = pd.merge(df_filtered, uncode[['name', 'alpha-3']], left_on='Country', right_on='name', how='left')

all_temp_filtered = all_temp_filtered.drop(columns=['name'])

print(all_temp_filtered)


          Country  AvgTemperature       Date alpha-3
0         Algeria        8.464158 2000-01-01     DZA
1       Argentina       25.356631 2000-01-01     ARG
2       Australia       20.979570 2000-01-01     AUS
3         Austria       -1.284946 2000-01-01     AUT
4         Bahamas       21.241935 2000-01-01     BHS
...           ...             ...        ...     ...
12219     Uruguay       20.059140 2011-12-01     URY
12220  Uzbekistan       -0.559140 2011-12-01     UZB
12221   Venezuela       25.627240 2011-12-01     NaN
12222     Vietnam       17.175627 2011-12-01     NaN
12223  Yugoslavia        4.612903 2011-12-01     NaN

[12224 rows x 4 columns]


# Data Analysis

1. The initial level

Use the choropleth chart to identify the country of interest at a glance, using the month of January 2000 as the starting point

In [ ]:

all_temp_filtered_2000 = all_temp_filtered[all_temp_filtered['Date'] == '2000-01-01']



# Create the choropleth map using plotly express
fig = px.choropleth(
    all_temp_filtered_2000,
    locations="alpha-3",  # ISO Alpha-3 codes for countries
    color="AvgTemperature",  # Column with temperature data
    hover_name="Country",  # Hover information (Country name)
    color_continuous_scale="Thermal",  # Yellow to Red color scale
    labels={"AvgTemperature": "Average Temperature (°C)", "Country": "Country"},  # Axis labels
    title="Average Temperature (°C) on January 1, 2000",  # Title of the map
)



# Show the map
fig.show()


Observations: The results are as expected, with the lower hemisphere being warmer in January, while the coldest area is in the northernmost part, i.e. Canada.

2. Pattern of temperature change from 2000 to 2011

Using line chart, while filter only the warmest and coldest countries, to see the similarities or differences in trend

In [ ]:


all_temp_filtered['Date'] = pd.to_datetime(all_temp_filtered['Date'])


dec_1_2011 = all_temp_filtered[all_temp_filtered['Date'] == '2011-12-01']


top_5_countries = dec_1_2011.nlargest(5, 'AvgTemperature')['Country']
bottom_5_countries = dec_1_2011.nsmallest(5, 'AvgTemperature')['Country']


selected_countries = pd.concat([top_5_countries, bottom_5_countries])


selected_data = all_temp_filtered[all_temp_filtered['Country'].isin(selected_countries)]


selected_data['Year'] = selected_data['Date'].dt.year
selected_data['Month'] = selected_data['Date'].dt.month

monthly_avg_temp = selected_data.groupby(['Country', 'Year', 'Month'])['AvgTemperature'].mean().reset_index()


monthly_avg_temp['Date'] = pd.to_datetime(monthly_avg_temp[['Year', 'Month']].assign(DAY=1))


fig = px.line(
    monthly_avg_temp,
    x='Date', 
    y='AvgTemperature', 
    color='Country', 
    title='Top 5 and Bottom 5 Countries by Avg Temperature on December 1, 2011',
    labels={'AvgTemperature': 'Average Temperature (°C)', 'Date': 'Date', 'Country': 'Country'}
)

fig.show()

C:\Users\patip\AppData\Local\Temp\ipykernel_17372\1606007675.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\patip\AppData\Local\Temp\ipykernel_17372\1606007675.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Observations: A warmer countries tend to be in near the equator and there's clearly less variance in their average temperatures. While the colder countries demonstrate a much more effect of seasonality. The pattern of seasonality is consistent, with no noticable differences between the year 2000 and 2011

3. Comparison on Changes in Temperature across Countries

In this part, comparison will be in two parts, first is the comparison of change during Winter (January) and second is Summer (June). 

In [ ]:

all_temp_filtered['Date'] = pd.to_datetime(all_temp_filtered['Date'])


temp_2000 = all_temp_filtered[all_temp_filtered['Date'] == '2000-01-01']
temp_2011 = all_temp_filtered[all_temp_filtered['Date'] == '2011-01-01']


merged_temp = pd.merge(
    temp_2000[['Country', 'AvgTemperature']], 
    temp_2011[['Country', 'AvgTemperature']], 
    on='Country', 
    suffixes=('_2000', '_2011')
)


merged_temp['Temp_Difference'] = merged_temp['AvgTemperature_2011'] - merged_temp['AvgTemperature_2000']


alpha_3_codes = all_temp_filtered[['Country', 'alpha-3']].drop_duplicates()


merged_temp = pd.merge(merged_temp, alpha_3_codes, on='Country', how='left')


merged_temp = merged_temp[['Country', 'alpha-3', 'AvgTemperature_2000', 'AvgTemperature_2011', 'Temp_Difference']]


print(merged_temp)


       Country alpha-3  AvgTemperature_2000  AvgTemperature_2011  \
0      Algeria     DZA             8.464158            10.860215   
1    Argentina     ARG            25.356631            24.546595   
2    Australia     AUS            20.979570            23.167025   
3      Austria     AUT            -1.284946             0.247312   
4      Bahamas     BHS            21.241935            21.818996   
..         ...     ...                  ...                  ...   
78     Uruguay     URY            22.421147            23.003584   
79  Uzbekistan     UZB             3.965950             1.641577   
80   Venezuela     NaN            24.878136            25.482079   
81     Vietnam     NaN            18.218638            12.528674   
82  Yugoslavia     NaN            -1.844086             0.471326   

    Temp_Difference  
0          2.396057  
1         -0.810036  
2          2.187455  
3          1.532258  
4          0.577061  
..              ...  
78         0.582437  
79     

In [104]:
import plotly.express as px

# Create the choropleth map with a diverging color scale centered around 0
fig = px.choropleth(
    merged_temp, 
    locations="alpha-3",  # Use 'Alpha-3' country codes for the map
    color="Temp_Difference",  # Color based on the temperature difference
    hover_name="Country",  # Show country name on hover
    color_continuous_scale="RdYlBu_r",  # Diverging color scale (you can use others like 'PiYG', 'RdGy', etc.)
    range_color=[-max(abs(merged_temp['Temp_Difference'])), max(abs(merged_temp['Temp_Difference']))],  # Set the color range symmetrically around 0
    labels={"Temp_Difference": "Temperature Difference (°C)", "Country": "Country"},
    title="Temperature Difference Between 1 Jan 2000 and 1 Jan 2011"
)

# Show the plot
fig.show()


Observations: There is no clear pattern of change / significant change that can be seen across countries.

Perform the same analysis, for Summer 

In [105]:


all_temp_filtered['Date'] = pd.to_datetime(all_temp_filtered['Date'])


temp_2000_june = all_temp_filtered[all_temp_filtered['Date'] == '2000-06-01']
temp_2011_june = all_temp_filtered[all_temp_filtered['Date'] == '2011-06-01']


merged_temp_june = pd.merge(
    temp_2000_june[['Country', 'AvgTemperature']], 
    temp_2011_june[['Country', 'AvgTemperature']], 
    on='Country', 
    suffixes=('_2000', '_2011')
)


merged_temp_june['Temp_Difference'] = merged_temp_june['AvgTemperature_2011'] - merged_temp_june['AvgTemperature_2000']

#
alpha_3_codes = all_temp_filtered[['Country', 'alpha-3']].drop_duplicates()


merged_temp_june = pd.merge(merged_temp_june, alpha_3_codes, on='Country', how='left')


fig = px.choropleth(
    merged_temp_june, 
    locations="alpha-3",  
    color="Temp_Difference",  
    hover_name="Country",  
    color_continuous_scale="RdYlBu_r",  # Diverging color scale (you can use others like 'PiYG', 'RdGy', etc.)
    range_color=[-max(abs(merged_temp['Temp_Difference'])), max(abs(merged_temp['Temp_Difference']))],  # Set the color range symmetrically around 0
    labels={"Temp_Difference": "Temperature Difference (°C)", "Country": "Country"},
    title="Temperature Difference Between 1 Jun 2000 and 1 Jun 2011"
)

# Show the plot
fig.show()

Observations: For Summer, it seems there's a slight clearer trend of higher temperature, albeit there's no clear spike aside for one outlier(Mexico).

# Conclusion

From the analysis, it seems that from the year 2000 to 2011, while there's a slight trend of higher temperature in Summer across the world, the rest of the findings are inconclusive, with no clear trend in comparison for winter, nor the clear pattern in time series chart.

Going forward, it might be useful for a broader comparison instead on comparing two data points. In addition, knowing how to eliminate outliers could also help.